In [50]:
import sqlalchemy
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import pyodbc

In [51]:
northwind = create_engine('sqlite:///Northwind_larges.sqlite')
northwind

Engine(sqlite:///Northwind_larges.sqlite)

In [52]:
chinook = create_engine('sqlite:///chinook.db')


In [53]:
frameShips = pd.read_sql_query("""select ShipVia, ShipName, ShipAddress as ShipAdress, ShipCity, ShipRegion,
ShipPostalCode, ShipCountry from Orders;""", 
    con=northwind.connect())

In [54]:
frameCustomer = pd.read_sql_query("""select Id as Customer_ID, CompanyName, ContactName, Region, City, Country from Customer;""", 
    con=northwind.connect())

In [55]:
frameProduct = pd.read_sql_query("""select CategoryId, ProductName as Product_Name,  UnitPrice, QuantityPerUnit, UnitsInStock
as UnitslnStock, Discontinued from Product ;""", 
    con=northwind.connect())

In [56]:
frameCategory = pd.read_sql_query("""select CategoryName as Category_Name, Description from Category;""", 
    con=northwind.connect())

In [57]:
fecha_inicio='2009-01-01'
fecha_fin='2030-12-31'
dataFrameDate = pd.DataFrame({'fecha': pd.date_range(fecha_inicio, fecha_fin)})
dataFrameDate['anio'] = dataFrameDate.fecha.dt.year
dataFrameDate['mes'] = dataFrameDate.fecha.dt.month
dataFrameDate['dia'] = dataFrameDate.fecha.dt.day
dataFrameDate['nombredia'] = dataFrameDate.fecha.dt.day_name()
dataFrameDate['diasemana'] = dataFrameDate.fecha.dt.dayofweek
dataFrameDate['semana'] = dataFrameDate.fecha.dt.weekofyear
dataFrameDate['trimestre'] = dataFrameDate.fecha.dt.quarter
dataFrameDate = dataFrameDate[['fecha', 'anio', 'mes', 'dia', 'nombredia', 'diasemana', 'semana', 'trimestre']] 


C:\Users\zambr\AppData\Local\Temp/ipykernel_14280/104617120.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  dataFrameDate['semana'] = dataFrameDate.fecha.dt.weekofyear


In [58]:
FrameEmployee = pd.read_sql_query("""select Employee.FirstName, Employee.LastName, Employee.Title, Employee.HireDate from Employee;""", con=northwind.connect())

In [88]:
frameTerritories = pd.read_sql_query("""select Territory.Id as Territory_ID,Territory.TerritoryDescription, 
Employee.Id as Employee_ID, Region.Id as Region_ID from Territory inner JOIN EmployeeTerritory 
on Territory.Id = EmployeeTerritory.TerritoryId INNER JOIN Employee
on EmployeeTerritory.EmployeeId = Employee.id INNER JOIN Region
on Territory.RegionId = Region.Id;""", con=northwind.connect())

In [92]:
FrameRegion = pd.read_sql_query("""select Region.RegionDescription as RegionDescription from region """, con=northwind.connect())

In [9]:
frameOrderSales = pd.read_sql_query("""SELECT Customer.Id as Customer_ID, EmployeeId as Employee_ID, ProductId as Products_ID, 
dim_ships.Id as Ship_ID, Dim_Time.TimeId as Time_ID, Orders.OrderDate, Orders.RequiredDate, Orders.ShippedDate as ShipeDate, 
OrderDetail.UnitPrice, OrderDetail.Quantity,
OrderDetail.Discount, (OrderDetail.UnitPrice * OrderDetail.Quantity) - OrderDetail.Discount as Total
FROM Product INNER JOIN OrderDetail
ON Product.Id = OrderDetail.ProductId INNER JOIN Orders
on OrderDetail.OrderId = Orders.Id INNER JOIN Employee
on Orders.EmployeeId = Employee.Id INNER JOIN Customer
on Orders.CustomerId = Customer.Id INNER JOIN dim_ships
on Orders.Id = dim_ships.Id INNER JOIN Dim_Time
ON Orders.OrderDate = substr(dim_time.fecha,0,11)""", 
    con=northwind.connect())

In [78]:
frameShips.fillna(method="bfill", inplace = True) 
frameCustomer.fillna(method="bfill", inplace = True) 
frameProduct.fillna(method="bfill", inplace = True) 
frameCategory.fillna(method="bfill", inplace = True) 
FrameEmployee.fillna(method="bfill", inplace = True) 
frameTerritories.fillna(method="bfill", inplace = True) 
FrameRegion.fillna(method="bfill", inplace = True) 
frameOrderSales.fillna(method="bfill", inplace = True) 
dataFrameDate.fillna(method="bfill", inplace = True) 

In [93]:
engine = sqlalchemy.create_engine('mssql+pyodbc://localhost/EsquemaNieveVALE?driver=SQL+Server+Native+Client+11.0')
try:     
    frameShips.to_sql(name='Dim_Ship', con=engine, if_exists='append',index=False)
    frameCustomer.to_sql('Dim_Customer', con=engine, if_exists='append',index=False)
    frameCategory.to_sql('Dim_Category', con=engine, if_exists='append',index=False)
    frameProduct.to_sql('Dim_Products', con=engine, if_exists='append',index=False)
    FrameRegion.to_sql('Dim_Region', con=engine, if_exists='append',index=False)
    frameTerritories.to_sql('Dim_Territories', con=engine, if_exists='append',index=False)
    FrameEmployee.to_sql('Dim_Employee', con=engine, if_exists='append',index=False)
    dataFrameDate.to_sql('Dim_Time', con=engine, if_exists='append',index=False)
    frameOrderSales.to_sql('Sales_Order', con=engine, if_exists='append',index=False)
except Exception as ex:
                 print("Ocurrió un error: {}".format(ex))